In [2]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from tabulate import tabulate
from matplotlib import pyplot as plt
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns
import functions as fun
import OLS_Estimation as OLS

In [3]:
df = pd.read_csv('Dataset.csv')
Nobs=df['ID'].count()
T = df['Year'].unique().size
df['const']=np.ones((Nobs,1))
estimation = OLS.get_coefficients_OLS(df)


model_labels = df['Model'].unique()
#j = 0
#for i in model_labels:
#    j = j
#    print(f'Car {j}: {i}')
#    j = j+1  

repeated_models = np.repeat(model_labels, T)
print(f' models: {len(model_labels)} \n repeated models: {len(repeated_models)}')




 models: 107 
 repeated models: 1177


In [4]:
#Manipuler for ms = 0 ikke har ssh for at blive væltet.
df_calc = df.copy()
df_calc = pd.get_dummies(df_calc, columns=['Segment'], drop_first=True)
df_calc = pd.get_dummies(df_calc, columns=['Year'], drop_first=True)
df_calc['China'] = (df_calc['Country'] == 'CH').astype(int)

df_dummies = df.copy()
df_dummies = pd.get_dummies(df_dummies, columns=['Segment'], drop_first=True)
df_dummies = pd.get_dummies(df_dummies, columns=['Year'], drop_first=True)
df_dummies['China'] = (df_dummies['Country'] == 'CH').astype(int)


print(f' df_calc shape: {df_calc.shape} \n df_dummies shape: {df_dummies.shape}')


 df_calc shape: (1177, 30) 
 df_dummies shape: (1177, 30)


In [13]:
x = df_calc[['const', 'Price', 'Range', 'HP', 'Chargetime']]
dummies = df_calc[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)

columns = X.columns
print(f' columns: {columns}')

X = np.array(X)
print(f' X shape: {X[:,2:].shape}\n X type {type(X)}\n')

p_j = df_calc['Price']
p_j = np.array(p_j)
print(f' p_j shape: {p_j.shape} \n p_j type: {type(p_j)}\n ')

alpha = estimation['Price']
alpha = np.array(alpha)
beta = estimation
beta = np.array(beta)
print(f' alpha {alpha} \n beta shape {beta.shape}')


 columns: Index(['const', 'Price', 'Range', 'HP', 'Chargetime', 'Segment_B', 'Segment_C',
       'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
       'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018',
       'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
       'China'],
      dtype='object')
 X shape: (1177, 21)
 X type <class 'numpy.ndarray'>

 p_j shape: (1177,) 
 p_j type: <class 'numpy.ndarray'>
 
 alpha -1.8694188591969425e-08 
 beta shape (23,)


In [6]:
ccp_list = fun.ccp(alpha, beta, X, p_j)

share_j: 1177
choice probability sum: 1.0 
 [array([[0.00105185]]), array([[0.00170497]]), array([[0.00072131]]), array([[0.00108618]]), array([[0.00088257]]), array([[0.00047232]]), array([[0.00031324]]), array([[0.00016207]]), array([[5.90524522e-05]]), array([[3.84629187e-05]]), array([[2.06153325e-05]])]


In [22]:
model_probability = pd.DataFrame({'Model': repeated_models, 'CCP': ccp_list})
print(f' shape: {model_probability.shape} \n model_probability: {model_probability}')


 shape: (1177, 2) 
 model_probability:      Model                         CCP
0       U5    [[0.001051848214382138]]
1       U5   [[0.0017049689903243955]]
2       U5   [[0.0007213083806897881]]
3       U5   [[0.0010861760177532146]]
4       U5   [[0.0008825698579298705]]
...    ...                         ...
1172  Free   [[3.365038549378552e-05]]
1173  Free   [[1.741044442080804e-05]]
1174  Free   [[6.377666176904705e-06]]
1175  Free   [[4.156253391234408e-06]]
1176  Free  [[2.2047173252693166e-06]]

[1177 rows x 2 columns]


In [23]:
probability_ratio = fun.probability_ratio(ccp_list[:10], repeated_models[:10])
print(f'shape: {probability_ratio.shape} \n probability_ratio: {probability_ratio}')

shape: (10, 10) 
 probability_ratio:                          U5                        U5  \
U5                  [[1.0]]      [[0.61693099425932]]   
U5   [[1.6209268286165266]]                   [[1.0]]   
U5    [[0.685753296746802]]   [[0.42306246317861096]]   
U5   [[1.0326357005713422]]    [[0.6370649694611475]]   
U5   [[0.8390657947242867]]    [[0.5176456949882405]]   
U5  [[0.44904176843671334]]   [[0.27702778466562483]]   
U5   [[0.2977994593312031]]   [[0.18372171653508704]]   
U5  [[0.15407909476074552]]   [[0.09505616912532272]]   
U5  [[0.05614160997617568]]   [[0.03463549926192101]]   
U5   [[0.0365669858282747]]  [[0.022559306924103975]]   

                          U5                        U5  \
U5    [[1.4582503718815167]]    [[0.9683957270184583]]   
U5     [[2.363717150622777]]    [[1.5696986146418253]]   
U5                   [[1.0]]    [[0.6640805623584238]]   
U5    [[1.5058413943762903]]                   [[1.0]]   
U5    [[1.2235680071897512]]     [[0.81254773

In [24]:
marginal_effects = fun.marginal_effects(ccp_list, repeated_models, columns, estimation)
print(f'shape: {marginal_effects.shape} \n marginal_effects: {marginal_effects}')

shape: (1177, 23) 
 marginal_effects:                             const                       Price  \
U5    [[-1.9642765926182664e-11]]    [[8.72309707538301e-06]]   
U5     [[-3.183948627798403e-11]]  [[1.4139502078114679e-05]]   
U5    [[-1.3470091491105509e-11]]   [[5.981892577286285e-06]]   
U5    [[-2.0283821353342524e-11]]   [[9.007781459589962e-06]]   
U5    [[-1.6481573002435592e-11]]   [[7.319252380013346e-06]]   
...                           ...                         ...   
Free   [[-6.284049699780201e-13]]   [[2.790664805746482e-07]]   
Free   [[-3.251317820885704e-13]]  [[1.4438679909484865e-07]]   
Free  [[-1.1909988737477552e-13]]  [[5.2890712191022474e-08]]   
Free   [[-7.761605845561697e-14]]   [[3.446828288140816e-08]]   
Free   [[-4.117204912412515e-14]]   [[1.828397195445162e-08]]   

                            Range                           HP  \
U5    [[-1.9366989904674923e-06]]  [[-3.6227379119108114e-05]]   
U5     [[-3.139247352603301e-06]]   [[-5.87219307

In [27]:
cross_marginal_effects = fun.cross_marginal_effects(ccp_list, estimation)
print(f'shape: {cross_marginal_effects.shape} \n cross_marginal_effects: {cross_marginal_effects}')

(1177, 1177, 22)
shape: (1177, 1177, 22) 
 cross_marginal_effects: [[[ 2.06829636e-14 -9.19104453e-09  2.03858382e-09 ...  3.65410692e-06
    4.34513099e-06  7.12915488e-07]
  [ 3.35255706e-14 -1.48980107e-08  3.30439521e-09 ...  5.92303994e-06
    7.04313940e-06  1.15558384e-06]
  [ 1.41834105e-14 -6.30278909e-09  1.39796558e-09 ...  2.50581586e-06
    2.97968790e-06  4.88884146e-07]
  ...
  [ 1.25406913e-16 -5.57280157e-11  1.23605354e-11 ...  2.21559287e-08
    2.63458117e-08  4.32261702e-09]
  [ 8.17262761e-17 -3.63173217e-11  8.05522205e-12 ...  1.44387698e-08
    1.71692695e-08  2.81700095e-09]
  [ 4.33523464e-17 -1.92648092e-11  4.27295594e-12 ...  7.65915910e-09
    9.10757412e-09  1.49430032e-09]]

 [[ 3.35255706e-14 -1.48980107e-08  3.30439521e-09 ...  5.92303994e-06
    7.04313940e-06  1.15558384e-06]
  [ 5.43424968e-14 -2.41485852e-08  5.35618285e-09 ...  9.60081434e-06
    1.14164136e-05  1.87311685e-06]
  [ 2.29902706e-14 -1.02163599e-08  2.26599991e-09 ...  4.06174416e-0

In [28]:
elasticity = fun.elasticity(ccp_list, repeated_models, columns, estimation, X)
print(f'shape: {elasticity.shape} \n elasticity: {elasticity}')

elasticity shape: 
(1177, 23)
shape: (1177, 23) 
 elasticity:          const                   Price                     Range  \
U5    [[-0.0]]   [[3.319415832878943]]   [[-0.3699656732746388]]   
U5    [[-0.0]]   [[3.317245570648046]]   [[-0.3697237865789487]]   
U5    [[-0.0]]  [[3.3205141873384756]]  [[-0.37008809043102225]]   
U5    [[-0.0]]   [[3.319301764642454]]   [[-0.3699529597930997]]   
U5    [[-0.0]]   [[3.319978329796893]]  [[-0.37002836639940584]]   
...        ...                     ...                       ...   
Free  [[-0.0]]   [[3.322799213676208]]  [[-0.37034276816651895]]   
Free  [[-0.0]]  [[3.3228531775555408]]   [[-0.3703487827136426]]   
Free  [[-0.0]]   [[3.322889838496069]]  [[-0.37035286876077506]]   
Free  [[-0.0]]   [[3.322897220053119]]  [[-0.37035369147261993]]   
Free  [[-0.0]]  [[3.3229037048338395]]     [[-0.37035441423421]]   

                           HP Chargetime                Segment_B Segment_C  \
U5    [[-1.1710560533349652]]   [[-0.0]]  

In [29]:
cross_elasticity = fun.cross_elasticity(ccp_list, estimation, X, repeated_models)

1177
Change in : U5 
 [[[ 0.         -0.0034952   0.00038956 ...  0.          0.
    0.00067777]]

 [[ 0.         -0.0034952   0.00038956 ...  0.          0.
    0.00067777]]

 [[ 0.         -0.0034952   0.00038956 ...  0.          0.
    0.00067777]]

 ...

 [[ 0.         -0.0034952   0.00038956 ...  0.          0.
    0.00067777]]

 [[ 0.         -0.0034952   0.00038956 ...  0.          0.
    0.00067777]]

 [[ 0.         -0.0034952   0.00038956 ...  0.          0.
    0.00067777]]]
Change in : U5 
 [[[ 0.         -0.00566546  0.00063144 ...  0.          0.
    0.00109862]]

 [[ 0.         -0.00566546  0.00063144 ...  0.          0.
    0.00109862]]

 [[ 0.         -0.00566546  0.00063144 ...  0.          0.
    0.00109862]]

 ...

 [[ 0.         -0.00566546  0.00063144 ...  0.          0.
    0.00109862]]

 [[ 0.         -0.00566546  0.00063144 ...  0.          0.
    0.00109862]]

 [[ 0.         -0.00566546  0.00063144 ...  0.          0.
    0.00109862]]]
Change in : U5 
 [[[ 0.  